In [6]:
from numpy import argmax
from keras.applications.inception_v3 import InceptionV3
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding
from keras.layers import Dropout
from keras.layers.merge import add
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.image import load_img, img_to_array
from nltk.translate.bleu_score import corpus_bleu

from keras.models import Sequential
from keras.layers import LSTM, Embedding, TimeDistributed, Dense, RepeatVector, Merge, Activation, Flatten
from keras.optimizers import Adam, RMSprop
from keras.layers.wrappers import Bidirectional


# define the CNN model
def defineCNNmodel():
	model = InceptionV3()
 	model.layers.pop()
 	model = Model(inputs=model.inputs, outputs=model.layers[-1].output)
 	#print(model.summary())
 	return model

# define the RNN model
def defineRNNmodel(vocab_size, max_len):
	embedding_size = 300
    # Input dimension is 2048 since we will feed it the encoded version of the image.
	image_model = Sequential([
        Dense(embedding_size, input_shape=(2048,), activation='relu'),
        RepeatVector(max_len)
    ])
    # Since we are going to predict the next word using the previous words(length of previous words changes with every iteration over the caption), we have to set return_sequences = True.
	caption_model  = Sequential([
        Embedding(vocab_size, embedding_size, input_length=max_len),
        LSTM(256, return_sequences=True),
        TimeDistributed(Dense(300))
    ])
    # Merging the models and creating a softmax classifier
	final_model = Sequential([
        Merge([image_model, caption_model], mode='concat', concat_axis=1),
        Bidirectional(LSTM(256, return_sequences=False)),
        Dense(vocab_size),
        Activation('softmax')
    ])
	final_model.compile(loss='categorical_crossentropy', optimizer=RMSprop(), metrics=['accuracy'])
	final_model.summary()
	return final_model


# map an integer to a word
def word_for_id(integer, tokenizer):
	for word, index in tokenizer.word_index.items():
		if index == integer:
			return word
	return None

# generate a description for an image, given a pre-trained model and a tokenizer to map integer back to word
def generate_desc(model, tokenizer, photo, max_length):
	# seed the generation process
	in_text = 'startseq'
	# iterate over the whole length of the sequence
	for i in range(max_length):
		# integer encode input sequence
		sequence = tokenizer.texts_to_sequences([in_text])[0]
		# pad input
		sequence = pad_sequences([sequence], maxlen=max_length)
		# predict next word
		yhat = model.predict([photo,sequence], verbose=0)
		# convert probability to integer
		yhat = argmax(yhat)
		# map integer to word
		word = word_for_id(yhat, tokenizer)
		# stop if we cannot map the word
		if word is None:
			break
		# append as input for generating the next word
		in_text += ' ' + word
		# stop if we predict the end of the sequence
		if word == 'endseq':
			break
	return in_text


def evaluate_model(model, photos, descriptions, tokenizer, max_length):
	actual, predicted = list(), list()
	
	for key, desc_list in descriptions.items():
		yhat = generate_desc(model, tokenizer, photos[key], max_length)
		references = [d.split() for d in desc_list]
		actual.append(references)
		predicted.append(yhat.split())

	print('BLEU-1: %f' % corpus_bleu(actual, predicted, weights=(1.0, 0, 0, 0)))
	print('BLEU-2: %f' % corpus_bleu(actual, predicted, weights=(0.5, 0.5, 0, 0)))
	print('BLEU-3: %f' % corpus_bleu(actual, predicted, weights=(0.3, 0.3, 0.3, 0)))
	print('BLEU-4: %f' % corpus_bleu(actual, predicted, weights=(0.25, 0.25, 0.25, 0.25)))

TabError: inconsistent use of tabs and spaces in indentation (<ipython-input-6-935e4452d1ee>, line 23)

In [7]:
import urllib.request

def tiny_url(url):
  apiurl = "http://tinyurl.com/api-create.php?url="
  tinyurl = urllib.request.urlopen(apiurl + url).read()
  return tinyurl.decode("utf-8")

url = input("Enter the url to short : ")

print(tiny_url(url))


Enter the url to short : http://localhost:8888/notebooks/Jupyter_Notebook/Untitled8.ipynb?kernel_name=python3
http://tinyurl.com/y7ewwcq5


In [16]:
from tkinter import *
import os

def register_user():
  username_info = username.get()
  password_info = password.get()

  file = open(username_info,"w")
  file.write(username_info + "\n")
  file.write(password_info)
  file.close()
  Label(screen1,text="registeration success",font="green",height="2",width="30").pack()

def register():
  global screen1
  screen1 = Toplevel(screen)
  screen1.geometry("350x250")
  screen1.title("register")
  global username
  global password
  global username_entry
  global password_entry
  username = StringVar()
  password = StringVar()
  Label(screen1,text="please enter the given information",height="2",width="30")
  Label(screen1,text="",height="2",width="30").pack()
  Label(screen1,text="username * ",height="2",width="30").pack()
  username_entry = Entry(screen1,textvariable = username)
  username_entry.pack()
  Label(screen1,text="password * ",height="2",width="30").pack()
  password_entry = Entry(screen1,textvariable = password)
  password_entry.pack()
  Label(screen1,text="",height="2",width="30").pack()
  Button(screen1,text="submit",height="2",width="30",command = register_user).pack()

def login_verify():
  username1 = username_verify.get()
  password1 = password_verify.get()
  list_of_dir = os.listdir()
  if username1 in list_of_dir:
    file = open(username1,"r")
    verify = file.read().splitlines()
    if password1 in verify:
      print("success")
    else:
      print("wrong password")
  
  else:
    print("username not found")

def login():
  global screen2
  global username_verify
  global password_verify
  username_verify = StringVar()
  password_verify = StringVar()

  screen2 = Toplevel(screen)
  screen2.geometry("500x300")
  screen2.title("login")
  Label(screen2,text="please enter the information for login",height="2",width="30").pack()
  Label(screen2,text="",height="2",width="30").pack()
  Label(screen2,text="username : ",height="2",width="30").pack()
  username_entry1 = Entry(screen2,textvariable = username_verify)
  username_entry1.pack()
  Label(screen2,text="password : ",height="2",width="30").pack()
  password_entry1 = Entry(screen2,textvariable = password_verify)
  password_entry1.pack()
  Label(screen2,text="",height="2",width="30").pack()
  Button(screen2,text="login",height="2",width="30",command=login_verify).pack()

def main_screen():
  global screen
  screen = Tk()
  screen.geometry("300x250")
  Label(text="instagram login").pack()
  Label(text="",height="2",width="30")
  Button(text="Login",height="2",width="30",bg="blue",command=login).pack()
  Label(text="",height="2",width="30").pack()
  Button(text="register",height="2",width="30",bg="green",command=register).pack()
  screen.mainloop()

main_screen()

username not found
username not found
username not found
username not found
username not found
username not found
username not found
username not found
username not found
username not found
username not found
username not found
username not found


In [18]:
#tic toc toe game using python 
from tkinter import Tk,ttk,Button
from tkinter import messagebox
from random import randint

ActivePlayer = 1
p1 = []
p2 = []
mov = 0

def SetLayout(id,player_symbol):
    if id==1:
        b1.config(text= player_symbol)
        b1.state(['disabled'])
    elif id==2:
        b2.config(text= player_symbol)
        b2.state(['disabled'])
    elif id==3:
        b3.config(text= player_symbol)
        b3.state(['disabled'])
    elif id==4:
        b4.config(text= player_symbol)
        b4.state(['disabled'])
    elif id==5:
        b5.config(text= player_symbol)
        b5.state(['disabled'])
    elif id==6:
        b6.config(text= player_symbol)
        b6.state(['disabled'])
    elif id==7:
        b7.config(text= player_symbol)
        b7.state(['disabled'])
    elif id==8:
        b8.config(text= player_symbol)
        b8.state(['disabled'])
    elif id==9:
        b9.config(text= player_symbol)
        b9.state(['disabled'])

def CheckWinner():
    global mov 
    winner = -1

    if(1 in p1) and (2 in p1) and (3 in p1):
        winner = 1
    if(1 in p2) and (2 in p2) and (3 in p2):
        winner = 2

    if(4 in p1) and (5 in p1) and (6 in p1):
        winner = 1
    if(4 in p2) and (5 in p2) and (6 in p2):
        winner = 2

    if(7 in p1) and (8 in p1) and (9 in p1):
        winner = 1
    if(7 in p2) and (8 in p2) and (9 in p2):
        winner = 2

    if(1 in p1) and (4 in p1) and (7 in p1):
        winner = 1
    if(1 in p2) and (4 in p2) and (7 in p2):
        winner = 2

    if(2 in p1) and (5 in p1) and (8 in p1):
        winner = 1
    if(2 in p2) and (5 in p2) and (8 in p2):
        winner = 2

    if(3 in p1) and (6 in p1) and ( 9 in p1):
        winner = 1
    if(3 in p2) and (6 in p2) and (9 in p2):
        winner = 2

    if(1 in p1) and (5 in p1) and ( 9 in p1):
        winner = 1
    if(1 in p2) and (5 in p2) and (9 in p2):
        winner = 2

    if(3 in p1) and (5 in p1) and ( 7 in p1):
        winner = 1
    if(3 in p2) and (5 in p2) and (7 in p2):
        winner = 2

    if winner ==1:
        messagebox.showinfo(title="Congratulations.", 
            message="Player 1 is the winner")
    elif winner ==2:
        messagebox.showinfo(title="Congratulations.", 
            message="Player 2 is the winner")
    elif mov ==9:
        messagebox.showinfo(title="Draw", 
            message="It's a Draw!!")

def ButtonClick(id):
    global ActivePlayer
    global p1,p2
    global mov

    if(ActivePlayer ==1):
        SetLayout(id,"X")
        p1.append(id)
        mov +=1
        root.title("Tic Tac Toe : Player 2")
        ActivePlayer =2

    elif(ActivePlayer==2):
        SetLayout(id,"O")
        p2.append(id)
        mov +=1
        root.title("Tic Tac Toe : Player 1")
        ActivePlayer =1
    CheckWinner()

def AutoPlay():
    global p1; global p2
    Empty = []
    for cell in range(9):
        if(not((cell +1 in p1) or (cell +1 in p2))):
            Empty.append(cell+1)
    try:
        RandIndex = randint(0,len(Empty) -1)
        ButtonClick(Empty[RandIndex])
    except:
        pass

def EnableAll():
    b1.config(text= " ")
    b1.state(['!disabled'])
    b2.config(text= " ")
    b2.state(['!disabled'])
    b3.config(text= " ")
    b3.state(['!disabled'])
    b4.config(text= " ")
    b4.state(['!disabled'])
    b5.config(text= " ")
    b5.state(['!disabled'])
    b6.config(text= " ")
    b6.state(['!disabled'])
    b7.config(text= " ")
    b7.state(['!disabled'])
    b8.config(text= " ")
    b8.state(['!disabled'])
    b9.config(text= " ")
    b9.state(['!disabled'])


def Restart():
    global p1,p2,mov,ActivePlayer
    p1.clear(); p2.clear()
    mov,ActivePlayer = 0,1
    root.title("Tic Tac Toe : Player 1")
    EnableAll()




root = Tk()
root.title("Tic Tac toe : Player 1")
st = ttk.Style()
st.configure("my.TButton", font=('Chiller',24,'bold'))

b1 = ttk.Button(root, text=" ", style="my.TButton")
b1.grid(row=1, column=0, sticky="nwse", ipadx=50,ipady=50)
b1.config(command = lambda : ButtonClick(1))


b2 = ttk.Button(root, text=" ",style ="my.TButton")
b2.grid(row=1, column=1, sticky="snew", ipadx=50, ipady=50)
b2.config(command = lambda : ButtonClick(2))

b3= ttk.Button(root, text=" ",style="my.TButton")
b3.grid(row=1, column=2, sticky="snew", ipadx=50,
        ipady=50)
b3.config(command = lambda : ButtonClick(3))

b4 = ttk.Button(root, text=" ",style="my.TButton")
b4.grid(row=2, column=0, sticky="snew", ipadx=50,
        ipady=50)
b4.config(command = lambda : ButtonClick(4))

b5 = ttk.Button(root, text=" ",style="my.TButton")
b5.grid(row=2, column=1, sticky="snew", ipadx=50,
        ipady=50)
b5.config(command = lambda : ButtonClick(5))

b6 = ttk.Button(root, text=" ",style="my.TButton")
b6.grid(row=2, column=2, sticky="snew", ipadx=50,
        ipady=50)
b6.config(command = lambda : ButtonClick(6))

b7 = ttk.Button(root, text=" ",style="my.TButton")
b7.grid(row=3, column=0, sticky="snew", ipadx=50,
        ipady=50)
b7.config(command = lambda : ButtonClick(7))

b8 = ttk.Button(root, text=" ",style="my.TButton")
b8.grid(row=3, column=1, sticky="snew", ipadx=50,
        ipady=50)
b8.config(command = lambda : ButtonClick(8))

b9 = ttk.Button(root, text=" ",style="my.TButton")
b9.grid(row=3, column=2, sticky="snew", ipadx=50,
        ipady=50)
b9.config(command = lambda : ButtonClick(9))

Button(text="New Game..", font=('Papyrus', 22, 'bold'), bg='Purple', fg='white',
            border=5, width=4,command = lambda :Restart()).grid(row=0, column=1, sticky="we")
root.resizable(0,0)
root.mainloop()

In [19]:
# Calculator using Python 
#                    Thanks to @papa_programmer

import tkinter as tk

cal = tk.Tk()
cal.title('Calculator')

def click(number):
  global oprator
  oprator = oprator + str(number)
  variable.set(oprator)

def clear():
  global oprator
  oprator = ""
  variable.set(oprator)

def sum():
  global oprator
  oprator = str(eval(oprator))
  variable.set(oprator)

oprator = ""

variable = tk.StringVar()
display = tk.Entry(cal,font=("arial",20,"bold"),bd=25,textvariable=variable,justify="right")
display.grid(columnspan=5)

button7 = tk.Button(cal,text="7",font=("arial",20,"bold"),bd=5,padx=10,command=lambda:click(7))
button7.grid(row=1,column=0)

button8 = tk.Button(cal,text="8",font=("arial",20,"bold"),bd=5,padx=10,command=lambda:click(8))
button8.grid(row=1,column=1)

button9 = tk.Button(cal,text="9",font=("arial",20,"bold"),bd=5,padx=10,command=lambda:click(9))
button9.grid(row=1,column=2)

button4 = tk.Button(cal,text="4",font=("arial",20,"bold"),bd=5,padx=10,command=lambda:click(4))
button4.grid(row=2,column=0)

button5 = tk.Button(cal,text="5",font=("arial",20,"bold"),bd=5,padx=10,command=lambda:click(5))
button5.grid(row=2,column=1)

button6 = tk.Button(cal,text="6",font=("arial",20,"bold"),bd=5,padx=10,command=lambda:click(6))
button6.grid(row=2,column=2)

button1 = tk.Button(cal,text="1",font=("arial",20,"bold"),bd=5,padx=10,command=lambda:click(1))
button1.grid(row=3,column=0)

button2 = tk.Button(cal,text="2",font=("arial",20,"bold"),bd=5,padx=10,command=lambda:click(2))
button2.grid(row=3,column=1)

button3 = tk.Button(cal,text="3",font=("arial",20,"bold"),bd=5,padx=10,command=lambda:click(3))
button3.grid(row=3,column=2)

button0 = tk.Button(cal,text="0",font=("arial",20,"bold"),bd=5,padx=10,command=lambda:click(0))
button0.grid(row=4,column=0)

buttonpoint = tk.Button(cal,text=" .",font=("arial",20,"bold"),bd=5,padx=10,command=lambda:click("."))
buttonpoint.grid(row=4,column=1)

buttonclear = tk.Button(cal,text="C",font=("arial",20,"bold"),bd=5,padx=10,command=clear)
buttonclear.grid(row=4,column=2)

buttonplus = tk.Button(cal,text="+",font=("arial",20,"bold"),bd=5,padx=10,command=lambda:click("+"))
buttonplus.grid(row=1,column=4)

buttonminus = tk.Button(cal,text="-",font=("arial",20,"bold"),bd=5,padx=10,command=lambda:click("-"))
buttonminus.grid(row=2,column=4)

buttonmulti = tk.Button(cal,text=" *",font=("arial",20,"bold"),bd=5,padx=10,command=lambda:click("*"))
buttonmulti.grid(row=3,column=4)

buttondiv = tk.Button(cal,text=" /",font=("arial",20,"bold"),bd=5,padx=10,command=lambda:click("/"))
buttondiv.grid(row=4,column=4)

buttonequal = tk.Button(cal,text="=",font=("arial",20,"bold"),bd=5,padx=10,command=sum)
buttonequal.grid(row=5,column=5)

cal.mainloop()

Exception in Tkinter callback
Traceback (most recent call last):
  File "/usr/lib/python3.6/tkinter/__init__.py", line 1705, in __call__
    return self.func(*args)
  File "<ipython-input-19-1d9662cd3fcb>", line 21, in sum
    oprator = str(eval(oprator))
  File "<string>", line 1
    012+1
      ^
SyntaxError: invalid token
